<a href="https://colab.research.google.com/github/engcarlo/TransferLearning-Datasets/blob/main/M3_Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Load Libraries

In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow
from google.colab import files

### Load External Image [Option I]

In [ ]:
uploaded = files.upload(target_dir = "downloads")
image_path = list(uploaded.keys())[0]

### Load Images from Repository [Option II]

In [ ]:
user = "engcarlo"
repo = "TransferLearning-Datasets"

# remove local directory if it already exists
if os.path.isdir(repo):
    !rm -rf {repo}

!git clone https://github.com/{user}/{repo}.git

In [ ]:
image_path = "/content/TransferLearning-Datasets/Dataset/Test Sample/sample01.jpg"

## Functions

In [ ]:
# Laoad Original Image
def LoadSampleIMG(image_path):
  try:
      image = cv2.imread(image_path)

      if image is not None:
          print("Image loaded successfully!")
          # Display the original image (optional)
          img_loaded = cv2_imshow(image)
      else:
          print("Error loading image. Check the file path.")

  except Exception as e:
      print(f"An error occurred: {e}")
  return img_loaded, image

# Convert to Grayscale
def Convert2Grayscale(image_path):
  image = cv2.imread(image_path)
  if image is not None:
      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      print("Image converted to grayscale successfully!")
      # Display the grayscale image (optional)
      img_grayscale = cv2_imshow(gray_image)
  else:
      img_grayscale = 0
      print("Could not convert the image to grayscale, as the original image was not loaded.")
  return img_grayscale, gray_image

# Convert to Black and White (Binary)
def Convert2Binary(image_path, colorSetup):
  image = cv2.imread(image_path)
  if image is not None:
      # Apply a threshold to binarize the image
      # The first return value is the threshold used (useful if using OTSU)
      # The second return value is the binarized image
      # cv2.THRESH_BINARY creates a binary image where pixels > threshold are set to max_value (255) and others to 0
      # You can experiment with different threshold values and threshold types (like cv2.THRESH_BINARY_INV, cv2.THRESH_TRUNC, etc.)
      gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      max_value = 255

      threshold_value = colorSetup['Threshold Value']
      threshold_type_str = colorSetup['Threshold Type']

      # Use a dictionary as a switch-case for threshold types
      threshold_types = {
          "THRESH_BINARY":        cv2.THRESH_BINARY,
          "THRESH_BINARY_INV":    cv2.THRESH_BINARY_INV,
          "THRESH_TRUNC":         cv2.THRESH_TRUNC,
          "THRESH_TOZERO":        cv2.THRESH_TOZERO,
          "THRESH_TOZERO_INV":    cv2.THRESH_TOZERO_INV,
          "THRESH_OTSU":          cv2.THRESH_OTSU
      }
      # Default to THRESH_BINARY if string is not recognized
      threshold_type = threshold_types.get(colorSetup['Threshold Type'], cv2.THRESH_BINARY)

      # If using OTSU, the colorSetup['Threshold Value'] is ignored and calculated automatically
      if threshold_types[colorSetup['Threshold Type']] == cv2.THRESH_OTSU:
          ret, binary_image = cv2.threshold(gray_image, 0, max_value, cv2.THRESH_OTSU)
      else:
          ret, binary_image = cv2.threshold(gray_image, colorSetup['Threshold Value'], max_value, threshold_types[colorSetup['Threshold Type']])

      print(f"Image binarized successfully using threshold: {ret}")
      # Display the binarized image (optional)
      img_binary = cv2_imshow(binary_image)
  else:
      print("Could not binarize the image, as the grayscale image was not generated.")
      return None
  return img_binary, binary_image

# Organize Images Side-by-Side (Original, Grayscale, and Black&White)
def display_images_side_by_side(img_original, img_gray, img_binary):
    """
    Displays original, grayscale, and binary images side by side horizontally.

    Args:
        img_original: The original image (RGB).
        img_gray: The grayscale image.
        img_binary: The binary image.
    """
    if img_original is None or img_gray is None or img_binary is None:
        print("One or more images are not available for display.")
        return

    # Ensure images have the same height for horizontal stacking
    height          = max(img_original.shape[0], img_gray.shape[0], img_binary.shape[0])
    width_original  = int(img_original.shape[1] * (height / img_original.shape[0]))
    width_gray      = int(img_gray.shape[1] * (height / img_gray.shape[0]))
    width_binary    = int(img_binary.shape[1] * (height / img_binary.shape[0]))

    img_original_resized  = cv2.resize(img_original, (width_original, height))
    img_gray_resized      = cv2.resize(img_gray, (width_gray, height))
    img_binary_resized    = cv2.resize(img_binary, (width_binary, height))

    # Convert grayscale and binary to 3 channels to stack with color image
    if len(img_gray_resized.shape) == 2:
        img_gray_resized    = cv2.cvtColor(img_gray_resized, cv2.COLOR_GRAY2BGR)
    if len(img_binary_resized.shape) == 2:
        img_binary_resized  = cv2.cvtColor(img_binary_resized, cv2.COLOR_GRAY2BGR)


    # Stack images horizontally
    combined_image = np.hstack(
        (img_original_resized, img_gray_resized, img_binary_resized)
        )
    img_combined = cv2_imshow(combined_image)

    return img_combined, combined_image

# Examples

### Original Image

In [ ]:
img_original, original_image = LoadSampleIMG(image_path)
img_original

### Grayscale Image Conversion

In [ ]:
img_grayscale, gray_image = Convert2Grayscale(image_path)
img_grayscale

### Binarize the image


How to adjust the threshold value for binarization
You can adjust the threshold value (`threshold_value` in the `cv2.threshold` function) by experimenting with different integer numbers between 0 and 255.

Additionally, `cv2.threshold` has different types of thresholding you can use:
- **cv2.THRESH_BINARY**: If the pixel is greater than the threshold, it becomes `max_value` (usually 255), otherwise it becomes 0.
- **cv2.THRESH_BINARY_INV**: The inverse of `cv2.THRESH_BINARY`.
- **cv2.THRESH_TRUNC**: If the pixel is greater than the threshold, it becomes the threshold, otherwise it remains the same.
- **cv2.THRESH_TOZERO**: If the pixel is greater than the threshold, it remains the same, otherwise it becomes 0.
- **cv2.THRESH_TOZERO_INV**: The inverse of `cv2.THRESH_TOZERO`.

To find the best threshold, you can:

1. **Experiment manually**: Try a few values and see the result.
2. **Use automatic methods**: OpenCV offers methods like OTSU's binarization (`cv2.THRESH_OTSU`) which automatically calculates the ideal threshold for images with two peaks in their histogram (e.g., foreground and background). To use OTSU, you pass 0 for the threshold value and add `cv2.THRESH_OTSU` to the threshold type.

You can modify the binarization code cell to experiment with different threshold values and types and see how they affect the resulting image.

In [ ]:
colorSetup = {}
colorSetup['Threshold Value'] = 100 # @param {type: "slider", min: 0, max: 255, step: 1}
colorSetup['Threshold Type']  = "THRESH_OTSU" # @param {type: "string"}["THRESH_BINARY", "THRESH_BINARY_INV", "THRESH_TRUNC", "THRESH_TOZERO", "THRESH_TOZERO_INV", "THRESH_OTSU"]

In [ ]:
img_binary, binary_image = Convert2Binary(image_path, colorSetup)
img_binary

### Compare original image with the binarized one

In [ ]:
img_combined, combined_images = display_images_side_by_side(original_image, gray_image, binary_image)
img_combined

### Save Image

In [ ]:
output_path = "combined_image.jpg"
cv2.imwrite(output_path, combined_images)